## Hard Hat Detector

### Initialize the Libraries

In [2]:
!pip install tensorflow==2.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 394.7 MB 17 kB/s 
     |████████████████████████████████| 132 kB 63.3 MB/s 
     |████████████████████████████████| 2.9 MB 47.8 MB/s 
     |████████████████████████████████| 14.8 MB 45.6 MB/s 
     |████████████████████████████████| 462 kB 60.6 MB/s 
     |████████████████████████████████| 3.8 MB 41.1 MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68721 sha256=017727de24a2e9810ee58b1e1b2be00b5f4ccbb67b93db36aca7321ded95fee4
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: numpy
    Found existing installation: n

In [3]:
!pip install opencv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement opencv (from versions: none)
ERROR: No matching distribution found for opencv


In [4]:
pip install keras==2.4.3 numpy==1.19.3 pillow==7.0.0 scipy==1.4.1 h5py==2.10.0 matplotlib==3.3.2 opencv-python keras-resnet==0.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.9 MB 6.7 MB/s 
     |████████████████████████████████| 2.1 MB 49.1 MB/s 
     |████████████████████████████████| 26.1 MB 1.4 MB/s 
     |████████████████████████████████| 11.6 MB 61.8 MB/s 
  Created wheel for keras-resnet: filename=keras_resnet-0.2.0-py2.py3-none-any.whl size=20486 sha256=ced69d3c1441189f8991bb93977127eb2fdb3841462eef035819513832dd967e
  Stored in directory: /root/.cache/pip/wheels/bd/ef/06/5d65f696360436c3a423020c4b7fd8c558c09ef264a0e6c575
Successfully built keras-resnet
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
  Attempting uninstall: pillow
    Found existing installation: 

In [6]:
pip install imageai --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 160 kB 4.2 MB/s 


In [7]:
import cv2 as cv
from imageai.Detection import ObjectDetection

import numpy as np
import requests as req
import os as os

### Show Window Function

In [8]:
def showImage(img):
    window_name = 'image'
    cv.imshow(window_name, img)
    cv.waitKey(0)
    cv.destroyAllWindows()

## Download Data

### Download Images of Hard Hats

In [10]:
hardhatLoc = 'http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n03492922'

hardhatImages = req.get(hardhatLoc).text
noOfImages = 0

if not os.path.exists('hardhat'):
    os.makedirs('hardhat')


In [12]:

for i in hardhatImages.split('\n'):
    
        r = req.get(i, timeout=0.5)
        file = i.split("/")[-1].split('\r')[0]
        if 'image/jpeg' in r.headers['Content-Type']:
            if len(r.content) > 8192:
                with open('hardhat\\' + file, 'wb') as outfile:
                    outfile.write(r.content)
                    noOfImages += 1
                    print('Success: ' + file)

        
print('*********** Download Finished **************')

MissingSchema: ignored

### Download Images of People

In [ ]:
peopleLoc = 'http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n07942152'

peopleImages = req.get(peopleLoc).text
noOfImages = 0

if not os.path.exists('people'):
    os.makedirs('people')

for i in peopleImages.split('\n'):
    try:
        r = req.get(i, timeout=0.5)
        file = i.split("/")[-1].split('\r')[0]
        if 'image/jpeg' in r.headers['Content-Type']:
            if len(r.content) > 8192:
                with open('people\\' + file, 'wb') as outfile:
                    outfile.write(r.content)
                    noOfImages += 1
                    print('Success: ' + file)
            else:
                print('Failed: ' + file + ' -- Image too small')
        else:
            print('Failed: ' + file + ' -- Not an image')

    except Exception as e:
        print('Failed: ' + file + ' -- Error')
        
print('*********** Download Finished **************')

### Download Pre-Train Models

In [ ]:
modelRetinaNet = 'https://github.com/OlafenwaMoses/ImageAI/releases/download/1.0/resnet50_coco_best_v2.0.1.h5'
modelYOLOv3 = 'https://github.com/OlafenwaMoses/ImageAI/releases/download/1.0/yolo.h5'
modelTinyYOLOv3 = 'https://github.com/OlafenwaMoses/ImageAI/releases/download/1.0/yolo-tiny.h5'

if not os.path.exists('yolo.h5'):
    r = req.get(modelYOLOv3, timeout=0.5)
    with open('yolo.h5', 'wb') as outfile:
        outfile.write(r.content)

## Train the Model

### Define the Detector

In [ ]:
detector = ObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath('yolo.h5')
detector.loadModel()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### Clean the Data

In [ ]:
hardhatImages = os.listdir("hardhat")
peopleOnly = detector.CustomObjects(person=True)

for i in hardhatImages:
    imageFile = "hardhat/{0}".format(i)
    detectedImage, detections = detector.detectCustomObjectsFromImage(custom_objects=peopleOnly, output_type="array",
                                                                      input_image=imageFile, 
                                                                      minimum_percentage_probability=30)
    if len(detections) < 0:
        os.remove(imageFile)

ValueError: Ensure you specified correct input image, input type, output type and/or output image path 

### Split the Data

In [ ]:
if not os.path.exists('hardhat/train/images'):
    os.makedirs('hardhat/train/images')
if not os.path.exists('hardhat/validation/images'):
    os.makedirs('hardhat/validation/images')

hardhatImages = os.listdir("hardhat")
hardhatTrainNums = round(len(hardhatImages) * 0.90)

for i in range(0, hardhatTrainNums):
    file = "hardhat/" + hardhatImages[i]
    if os.path.isfile(file):
        os.rename(file, "hardhat/train/images/" + hardhatImages[i])
    
hardhatImages = os.listdir("hardhat")

for i in hardhatImages:
    file = "hardhat/" + i
    if os.path.isfile(file):
        os.rename(file, "hardhat/validation/images/" + i)

### Train the Model

In [ ]:
from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="hardhat")

trainer.setTrainConfig(object_names_array=["person hardhat"], batch_size=4, num_experiments=200, 
                       train_from_pretrained_model="yolo.h5")

trainer.trainModel()

Generating anchor boxes for training images and annotation...
Average IOU for 9 anchors: 0.75
Anchor Boxes generated.
Detection configuration saved in  hardhat\json\detection_config.json
Training on: 	['person hardhat']
Training with Batch Size:  4
Number of Experiments:  200
Training with transfer learning from pretrained Model


E:\Development\Anaconda\envs\ImageAI\lib\site-packages\keras\callbacks\callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Epoch 1/200
944/944 [==============================] - 856s 907ms/step - loss: 36.9858 - yolo_layer_16_loss: 7.0103 - yolo_layer_17_loss: 10.9458 - yolo_layer_18_loss: 19.0297 - val_loss: 22.8127 - val_yolo_layer_16_loss: 5.6805 - val_yolo_layer_17_loss: 6.6321 - val_yolo_layer_18_loss: 12.3109

Epoch 2/200
944/944 [==============================] - 774s 820ms/step - loss: 20.2127 - yolo_layer_16_loss: 4.6473 - yolo_layer_17_loss: 5.6500 - yolo_layer_18_loss: 9.9154 - val_loss: 12.3263 - val_yolo_layer_16_loss: 5.5798 - val_yolo_layer_17_loss: 6.8109 - val_yolo_layer_18_loss: 11.0726
Epoch 3/200
944/944 [==============================] - 776s 822ms/step - loss: 18.3042 - yolo_layer_16_loss: 4.7082 - yolo_layer_17_loss: 4.9417 - yolo_layer_18_loss: 8.6543 - val_loss: 30.1679 - val_yolo_layer_16_loss: 3.5330 - val_yolo_layer_17_loss: 5.7483 - val_yolo_layer_18_loss: 10.3959
Epoch 4/200
944/944 [==============================] - 772s 817ms/step - loss: 15.8019 - yolo_layer_16_loss: 3.5019

KeyboardInterrupt: 

### Evaluate the Model

In [ ]:
model05 = trainer.evaluateModel(model_path="hardhat\models\detection_model-ex-005--loss-0014.238.h5", 
                      json_path="hardhat\json\detection_config.json", iou_threshold=0.5, 
                      object_threshold=0.3, nms_threshold=0.5)
model10 = trainer.evaluateModel(model_path="hardhat\models\detection_model-ex-010--loss-0011.053.h5", 
                      json_path="hardhat\json\detection_config.json", iou_threshold=0.5, 
                      object_threshold=0.3, nms_threshold=0.5)
model15 = trainer.evaluateModel(model_path="hardhat\models\detection_model-ex-015--loss-0009.620.h5", 
                      json_path="hardhat\json\detection_config.json", iou_threshold=0.5, 
                      object_threshold=0.3, nms_threshold=0.5)
model20 = trainer.evaluateModel(model_path="hardhat\models\detection_model-ex-020--loss-0008.462.h5", 
                      json_path="hardhat\json\detection_config.json", iou_threshold=0.5, 
                      object_threshold=0.3, nms_threshold=0.5)

print('---------------------------------------------------------')
print('Iteration 05:', model05[0]['average_precision']['person hardhat'],
     'Iteration 10:', model10[0]['average_precision']['person hardhat'],
     'Iteration 15:', model15[0]['average_precision']['person hardhat'],
     'Iteration 20:', model20[0]['average_precision']['person hardhat'])
print('---------------------------------------------------------')

Starting Model evaluation....
Model File:  hardhat\models\detection_model-ex-005--loss-0014.238.h5 

Using IoU :  0.5
Using Object Threshold :  0.3
Using Non-Maximum Suppression :  0.5
person hardhat: 0.7165
mAP: 0.7165
Starting Model evaluation....
Model File:  hardhat\models\detection_model-ex-010--loss-0011.053.h5 

Using IoU :  0.5
Using Object Threshold :  0.3
Using Non-Maximum Suppression :  0.5
person hardhat: 0.7865
mAP: 0.7865
Starting Model evaluation....
Model File:  hardhat\models\detection_model-ex-015--loss-0009.620.h5 

Using IoU :  0.5
Using Object Threshold :  0.3
Using Non-Maximum Suppression :  0.5
person hardhat: 0.8355
mAP: 0.8355
Starting Model evaluation....
Model File:  hardhat\models\detection_model-ex-020--loss-0008.462.h5 

Using IoU :  0.5
Using Object Threshold :  0.3
Using Non-Maximum Suppression :  0.5
person hardhat: 0.8446
mAP: 0.8446
---------------------------------------------------------
Iteration 05: 0.7164811838244572 Iteration 10: 0.7864502281527

### Random Hard Hat Test

In [ ]:
from imageai.Detection.Custom import CustomObjectDetection

detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath("hardhat\models\detection_model-ex-020--loss-0008.462.h5")
detector.setJsonPath("hardhat\json\detection_config.json")
detector.loadModel()

person hardhat  :  48.9349365234375  :  [198, 16, 253, 127]
--------------------------------
person hardhat  :  92.02308058738708  :  [84, 71, 127, 114]
--------------------------------


In [ ]:
import random

testImages = os.listdir("hardhat/validation/images")
randomFile = testImages[random.randint(0, len(testImages) - 1)]

detectedImage, detections = detector.detectObjectsFromImage(output_type="array", 
                                                            input_image="hardhat/validation/images/{0}".format(randomFile), 
                                                            minimum_percentage_probability=30)
showImage(detectedImage)

for eachObject in detections:
    print(eachObject["name"] , " : ", eachObject["percentage_probability"], " : ", eachObject["box_points"] )
    print("--------------------------------")

person hardhat  :  81.50655627250671  :  [137, 74, 194, 119]
--------------------------------
person hardhat  :  64.44770097732544  :  [290, 89, 328, 123]
--------------------------------
person hardhat  :  70.66482901573181  :  [252, 90, 290, 130]
--------------------------------
person hardhat  :  54.37401533126831  :  [358, 93, 385, 131]
--------------------------------
person hardhat  :  59.80772376060486  :  [125, 100, 145, 136]
--------------------------------
person hardhat  :  45.671346783638  :  [222, 101, 249, 130]
--------------------------------
person hardhat  :  77.1479070186615  :  [314, 99, 361, 142]
--------------------------------
